## Deliverable 2. Create a Customer Travel Destinations Map.

In [1]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [2]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("Weather_Database/WeatherPy_database.csv")
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,Marawi,7.9986,124.2928,69.66,86,99,5.79,PH,2022-02-11 07:06:50
1,1,Kisangani,0.5167,25.2000,83.79,57,73,4.18,CD,2022-02-11 07:06:51
2,2,Punta Arenas,-53.1500,-70.9167,46.51,71,75,11.50,CL,2022-02-11 07:03:37
3,3,Albany,42.6001,-73.9662,34.38,90,93,2.10,US,2022-02-11 07:06:51
4,4,Svecha,58.2785,47.5164,21.69,94,100,7.18,RU,2022-02-11 07:06:52


In [3]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 75
What is the maximum temperature you would like for your trip? 90


In [4]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
1,1,Kisangani,0.5167,25.2000,83.79,57,73,4.18,CD,2022-02-11 07:06:51
6,6,Atuona,-9.8000,-139.0333,78.17,76,79,17.74,PF,2022-02-11 07:06:53
10,10,Paciran,-6.8767,112.3761,83.05,75,87,10.09,ID,2022-02-11 07:02:22
11,11,Vila Velha,-20.3297,-40.2925,75.15,91,75,6.91,BR,2022-02-11 07:06:54
15,15,Iquitos,-3.7481,-73.2472,75.24,100,75,0.00,PE,2022-02-11 07:05:24
18,18,Paidha,2.4167,30.9855,77.76,29,100,4.16,UG,2022-02-11 07:06:56
20,20,Rikitea,-23.1203,-134.9692,79.34,74,14,17.67,PF,2022-02-11 07:06:57
23,23,Road Town,18.4167,-64.6167,75.00,78,13,23.26,VG,2022-02-11 07:02:23
32,32,Sibolga,1.7427,98.7792,84.34,68,100,4.32,ID,2022-02-11 07:07:02
33,33,Kilindoni,-7.9139,39.6668,81.70,75,100,10.92,TZ,2022-02-11 07:07:03


In [5]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.count()

City_ID       173
City          173
Lat           173
Lng           173
Max Temp      173
Humidity      173
Cloudiness    173
Wind Speed    173
Country       173
Date          173
dtype: int64

In [6]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
clean_df = preferred_cities_df.dropna()

In [7]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_df[["City", "Country", "Max Temp", "Humidity","Cloudiness","Wind Speed","Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Humidity,Cloudiness,Wind Speed,Lat,Lng,Hotel Name
1,Kisangani,CD,83.79,57,73,4.18,0.5167,25.2000,
6,Atuona,PF,78.17,76,79,17.74,-9.8000,-139.0333,
10,Paciran,ID,83.05,75,87,10.09,-6.8767,112.3761,
11,Vila Velha,BR,75.15,91,75,6.91,-20.3297,-40.2925,
15,Iquitos,PE,75.24,100,75,0.00,-3.7481,-73.2472,
18,Paidha,UG,77.76,29,100,4.16,2.4167,30.9855,
20,Rikitea,PF,79.34,74,14,17.67,-23.1203,-134.9692,
23,Road Town,VG,75.00,78,13,23.26,18.4167,-64.6167,
32,Sibolga,ID,84.34,68,100,4.32,1.7427,98.7792,
33,Kilindoni,TZ,81.70,75,100,10.92,-7.9139,39.6668,


In [8]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    params["location"] = f"{lat},{lng}"
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")
        

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [9]:
# 7. Drop the rows where there is no Hotel Name.
clean_hotel_df = hotel_df.dropna()

In [10]:
# 8a. Create the output File (CSV)
output_data_file = "Weather_Database/WeatherPy_vacation.csv"
# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [11]:
clean_hotel_df.head()

,City,Country,Max Temp,Humidity,Cloudiness,Wind Speed,Lat,Lng,Hotel Name
1,Kisangani,CD,83.79,57,73,4.18,0.5167,25.2000,Grand Ramela Hotel
6,Atuona,PF,78.17,76,79,17.74,-9.8000,-139.0333,Villa Enata
10,Paciran,ID,83.05,75,87,10.09,-6.8767,112.3761,Tanjung Kodok Beach Resort
11,Vila Velha,BR,75.15,91,75,6.91,-20.3297,-40.2925,Hotel Vitória Palace
15,Iquitos,PE,75.24,100,75,0.00,-3.7481,-73.2472,Cortez Malaga Techy Caroly


In [14]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>City Name</dt><dd>{City}</dd>
<dt>Country Code</dt><dd>{Country}</dd>
<dt>Humidity</dt><dd>{Humidity} %</dd>
<dt>Cloudiness</dt><dd>{Cloudiness} %</dd>
<dt>Wind Speed</dt><dd>{Wind Speed} mph</dd>
<dt>Maximum Temp</dt><dd>{Max Temp}</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [16]:
# 11a. Add a marker layer for each city to the map. 
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(marker_layer)
# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))